In [3]:
pip install bs4 pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install requests beautifulsoup4 pandas



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install psycopg2-binary



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_pages(start_page, end_page):
    base_url = 'https://www.buyrentkenya.com/houses-for-sale'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    properties = []

    for page_num in range(start_page, end_page + 1):
        url = f'{base_url}?page={page_num}'  
        print(f"Scraping page {page_num}: {url}")
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
            continue  

        soup = BeautifulSoup(response.content, 'html.parser')

        listings = soup.find_all('div', class_='listing-card')  

        for listing in listings:
            # Extract title
            title_tag = listing.find('h2')
            title = title_tag.get_text(strip=True) if title_tag else 'No title'

            # Extract price
            price_tag = listing.find('p', class_='text-xl font-bold leading-7 text-grey-900')
            price = price_tag.get_text(strip=True) if price_tag else 'No price'

            # Extract location
            location_tag = listing.find('p', class_='ml-1 truncate text-sm font-normal capitalize text-grey-650')
            location = location_tag.get_text(strip=True) if location_tag else 'No location'

            # Swiper slides extraction (bedrooms, bathrooms, size)
            swiper_div = listing.find('div', class_='scrollable-list')
            bedrooms = bathrooms = size = 'N/A'
            if swiper_div:
                slides = swiper_div.find_all('div', class_='swiper-slide')
                for slide in slides:
                    text = slide.get_text(strip=True)
                    if 'Bedroom' in text:
                        bedrooms = text
                    elif 'Bathroom' in text:
                        bathrooms = text
                    elif 'm²' in text:
                        size = text

            # Append the data to the list
            properties.append({
                'Title': title,
                'Price': price,
                'Location': location,
                'Bedrooms': bedrooms,
                'Bathrooms': bathrooms,
                'Size': size
            })

    # Convert to DataFrame
    df = pd.DataFrame(properties)
    return df

In [8]:
df_all_pages = scrape_pages(start_page=1, end_page=4)

Scraping page 1: https://www.buyrentkenya.com/houses-for-sale?page=1
Failed to retrieve the page. Status code: 404
Scraping page 2: https://www.buyrentkenya.com/houses-for-sale?page=2
Scraping page 3: https://www.buyrentkenya.com/houses-for-sale?page=3
Scraping page 4: https://www.buyrentkenya.com/houses-for-sale?page=4


In [9]:
df_all_pages.head()

,Title,Price,Location,Bedrooms,Bathrooms,Size
0,4 Bed House with En Suite at Manyanja Road,"KSh 9,500,000","4, Manyanja Road, Donholm",4 Bedrooms,3 Bathrooms,225 m²
1,4 Bed Villa with En Suite in Westlands Area,"KSh 96,000,000","Westlands Area, Westlands",4 Bedrooms,N/A,N/A
2,5 Bed Townhouse with En Suite in Lavington,"KSh 130,000,000",Lavington,5 Bedrooms,6 Bathrooms,N/A
3,5 Bed Villa with En Suite in Lavington,"KSh 75,000,000",Lavington,5 Bedrooms,6 Bathrooms,N/A
4,4 Bed Townhouse with En Suite in Karen,"KSh 64,000,000",Karen,4 Bedrooms,4 Bathrooms,N/A


In [10]:
from sqlalchemy import create_engine
import pandas as pd
import seaborn as sns
import datetime

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
    "postgresql+psycopg2://postgres:@localhost:postgres")



In [12]:
df_all_pages.to_sql('Real_Estate', engine, if_exists='replace', index=False, schema='housing')  # If schema exists


75